# Recognize Faces in Real Time (Webcam)
This notebook uses the trained face recognition model to recognize faces in real-time using a webcam.

In [3]:
# Import Required Libraries
import cv2
import face_recognition
import pickle
import time

In [4]:
# Load encodings
with open("../model/encodings.pickle", "rb") as f:
    data = pickle.load(f)

In [5]:
# Initialize Webcam
video = cv2.VideoCapture(0)

# Time tracking
prev_time = 0

while True:
    ret, frame = video.read()
    if not ret:
        break

    # Time at start of this frame
    curr_time = time.time()

    # Calculate FPS
    fps = 1 / (curr_time - prev_time) if prev_time != 0 else 0
    prev_time = curr_time

    # Convert to RGB for face_recognition
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Face detection and recognition
    boxes = face_recognition.face_locations(rgb)
    encodings = face_recognition.face_encodings(rgb, boxes)
    names = []

    for encoding in encodings:
        matches = face_recognition.compare_faces(data["encodings"], encoding)
        name = "Unknown"

        if True in matches:
            matched_idxs = [i for (i, b) in enumerate(matches) if b]
            counts = {}
            for i in matched_idxs:
                name = data["names"][i]
                counts[name] = counts.get(name, 0) + 1
            name = max(counts, key=counts.get)
        names.append(name)

    # Draw boxes and labels
    for ((top, right, bottom, left), name) in zip(boxes, names):
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
        cv2.putText(frame, name, (left, top - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 255, 0), 2)

    # Draw FPS
    cv2.putText(frame, f"FPS: {fps:.2f}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

    # Show the frame
    cv2.imshow("Face Recognition", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

video.release()
cv2.destroyAllWindows()